<a href="https://colab.research.google.com/github/tarod13/Stochastic_Games/blob/master/stochastic_games_herkovitz_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import itertools
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter

device = 'cpu'

# Game Class Definition

First, we will define a general class for stochastic games that requires the tuple $\langle N,S,A,R,T,\beta \rangle$ to define each game.

In [23]:
class multi_player_game(nn.Module):
  def __init__(self, N, S, A, R, T, beta, device):
    super().__init__()

    # Store game defining parameters
    self.N = N
    self.S_no_string = S.copy()
    self.S = [str(s) for s in S]
    self.A = A.copy()
    self.R = {}
    for player in self.players():
      self.R[player] = {}
      for s_key, val in R[player].items():
        self.R[player][s_key] = val.clone().to(device)
    self.transition_map = T[0]
    self.transition_type = T[1]
    self.beta = beta
    self.device = device

    # Define useful constants
    self.N_S = len(S)    
    self.N_A_S_vector = {'1':[], '2':[]}
    self.N_A_S_reduced_vector = {'1':[], '2':[]}
    for player in self.players():
      for s in self.S:
        n_actions_player_in_s = A[player][s]      
        self.N_A_S_vector[player].append(n_actions_player_in_s)
        if self.more_than_one_action(player, s):
          self.N_A_S_reduced_vector[player].append(n_actions_player_in_s)
    
    self.N_A_total = {}
    self.N_A_reduced = {}
    self.N_S_reduced = {}
    for player in self.players():
      self.N_A_S_vector[player] = torch.FloatTensor(self.N_A_S_vector[player]).to(device).view(-1,1)
      self.N_A_S_reduced_vector[player] = torch.FloatTensor(self.N_A_S_reduced_vector[player]).to(device).view(-1,1)
      self.N_A_total[player] = int(self.N_A_S_vector[player].sum().item())
      self.N_A_reduced[player] = int(self.N_A_S_reduced_vector[player].sum().item())
      self.N_S_reduced[player] = self.N_A_S_reduced_vector[player].view(-1).shape[0]

    self.n_restrictions = 0
    for player in self.players():
      self.n_restrictions += self.N_A_total[player] + self.N_A_reduced[player] + self.N_S_reduced[player]
    self.n_vars = 2*self.N_S + self.N_A_reduced['1'] + self.N_A_reduced['2']

    # Define useful variables
    self.n_save = 0

    # Define policy and value parameter tensors
    self.pi = {}
    for player in self.players():
      self.pi[player] = nn.ParameterDict()
      for s in self.S: 
        n_actions_player_in_s = A[player][s]
        self.pi[player][s] = Parameter(torch.Tensor(n_actions_player_in_s,1))
        nn.init.constant_(self.pi[player][s], 1.0/(n_actions_player_in_s+1))
    
    self.v = Parameter(torch.Tensor(self.N_S,2))
    nn.init.zeros_(self.v)

  def forward(self):
    return self.pi, self.v
  
  #-----------------------------------------------------------------------------
  # Useful methods
  def get_state_index(self, state):
    if state in self.S:
      return self.S.index(state)
    elif state in self.S_no_string:
      return self.S_no_string.index(state)
    else:
      assert 0 == 1, 'Invalid state'

  # TODO: consider more than 2 players (is this method really necessary?)
  @staticmethod 
  def player_dim(i):
    if i in [0, '1']:
      return 'i'
    elif i in [1, '2']:
      return 'j'
    else:
      assert 0 == 1, 'Invalid player id'

  def get_player_id(self, player):
    player_id = int(player)-1
    if player_id >= 0 and player_id < self.N:
      return player_id
    else:
      assert 0 == 1, 'Invalid player'
  
  # TODO: consider more than 2 players (is this method really necessary?)
  @staticmethod
  def other_player(i):
    if i == 1:
      return 2
    elif i == 2:
      return 1
    elif i == '1':
      return '2'
    elif i == '2':
      return '1'
    else:
      assert 0 == 1, 'Invalid player id'

  def players_id(self):
    return range(0, self.N)

  def players(self):
    return iter([str(player_id+1) for player_id in self.players_id()]) 

  def state_player_id_pairs(self):
    return itertools.product(self.S, self.players_id())

  def state_player_pairs(self):
    return itertools.product(self.S, self.players())

  def player_consistent_reward_matrices(self):
    consistent_R = {'1':{}, '2':{}}
    for s in self.S:
      consistent_R['1'][s] = self.R['1'][s].clone()
      consistent_R['2'][s] = torch.t(self.R['2'][s].clone())  
    return consistent_R 

  def more_than_one_action(self, player, s):
    if self.A[player][s] > 1:
      return True
    else:
      return False
  
  def pi2vec(self):
    pi_vector = {}
    for player in self.players():
      pi_list = []
      for s in self.S:
        if self.more_than_one_action(player, s):
          pi_list.append(self.pi[player][s])
      pi_vector[player] = torch.cat(pi_list, dim=0)
    return pi_vector
  
  def pi_sum(self):
    sum_vector = {}
    for player in self.players():
      sum_list = []
      for s in self.S:
        if self.more_than_one_action(player, s):
          sum_list.append(self.pi[player][s].sum(0, keepdim=True))        
      sum_vector[player] = torch.cat(sum_list, dim=0)
    return sum_vector

  def copy_game(self):
    Ttuple = (self.transition_map, self.transition_type)
    game_copy = multi_player_game(self.N, self.S, self.A, self.R, Ttuple, self.beta, self.device).to(device)
    game_copy.load_state_dict(self.state_dict())
    return game_copy
  
  def save(self):
    torch.save(self.state_dict(), './game_save_'+str(self.n_save)+'.pth')
    print('Game '+str(self.n_save)+' saved succesfully')
    self.n_save += 1
  
  #-----------------------------------------------------------------------------
  # Methods related with the transition matrices

  def transition_matrix(self): # TODO: consider other 2 cases
    transition_matrix = torch.zeros((self.N_S,self.N_S)).to(self.device)
    for state in self.S_no_string:
      strategy_1 = self.pi['1'][str(state)]
      strategy_2 = self.pi['2'][str(state)]

      det, dep = self.transition_type[str(state)]
      N_A1 = self.A['1'][str(state)]
      N_A2 = self.A['2'][str(state)]
      id_s = self.get_state_index(state)

      if det and dep:
        for a1 in range(0,N_A1):
          for a2 in range(0,N_A2):
            _, next_state = self.transition_map(state, [a1,a2])
            id_ns = self.get_state_index(next_state)
            transition_prob = strategy_1[a1,0] * strategy_2[a2,0]
            transition_matrix[id_s, id_ns] = transition_matrix[id_s, id_ns] + transition_prob
      elif (not det) and (not dep):
        _, transition_dic = self.transition_map(state, [])
        for next_state, transition_prob in transition_dic:
          transition_matrix[id_s, self.get_state_index(next_state)] = transition_prob
    return transition_matrix

  def partial_transition_matrices(self): # TODO: consider other 2 cases
    # Create dictionary of transition matrices for each state given
    # the strategy of the other player 
    transition_matrices = {}
    for state in self.S_no_string:
      strategy_1 = self.pi['1'][str(state)]
      strategy_2 = self.pi['2'][str(state)]
      N_A1 = self.A['1'][str(state)]
      N_A2 = strategy_2.shape[0]
      transition_matrices[str(state)] = {
          '1': torch.zeros((N_A1,self.N_S)).to(self.device),
          '2': torch.zeros((N_A2,self.N_S)).to(self.device)
      }

      # Fill matrices with transition probabilities depending on the type
      # of transition, i.e., if deterministic or random and independent or
      # not on the actions
      det, dep = self.transition_type[str(state)]
      if det and dep:
        for a1 in range(0,N_A1):
          for a2 in range(0,N_A2):
            _, next_state = self.transition_map(state, [a1,a2])
            id_ns = self.get_state_index(next_state)
            transition_prob1 = strategy_2[a2,0]
            transition_prob2 = strategy_1[a1,0]
            transition_matrices[str(state)]['1'][a1, id_ns] = (
                transition_matrices[str(state)]['1'][a1, id_ns] + transition_prob1)
            transition_matrices[str(state)]['2'][a2, id_ns] = (
                transition_matrices[str(state)]['2'][a2, id_ns] + transition_prob2)
      elif (not det) and (not dep):
        _, transition_dic = self.transition_map(state, [])
        for next_state, transition_prob in transition_dic:
          transition_matrices[str(state)]['1'][:, self.get_state_index(next_state)] = transition_prob
          transition_matrices[str(state)]['2'][:, self.get_state_index(next_state)] = transition_prob
    return transition_matrices

  #-----------------------------------------------------------------------------
  # Methods related with the expected reward and value function calculation

  # TODO: consider more than 2 players  
  def expected_reward(self):
    r_mean = torch.zeros((self.N_S,2)).to(self.device)
    for player_id in self.players_id():
      player = str(player_id+1)
      RM_i = self.R[player]
      for s in self.S:
        strategy_1 = self.pi['1'][s]
        strategy_2 = self.pi['2'][s]
        r_mean_1 = torch.einsum('ij,ik->jk', RM_i[s], strategy_1)
        r_mean[self.get_state_index(s), player_id] = (r_mean_1 * strategy_2).sum()
    return r_mean
  
  def partial_expected_reward(self):
    # Create reward dictionary for each combination of players
    r_mean = {}
    for player in self.players():  
      r_mean[player] = {'1':{}, '2':{}}

    # Calculate expected reward for combination of players wrt the policy of one of the players
    for s, player in self.state_player_pairs():
      other_player = self.other_player(player) # Player used to calculate expected reward
      for second_player in self.players():  
        N_A = self.A[second_player][s]
        RM_i = self.R[second_player] # Reward matrix for one of the players  
        strategy = self.pi[other_player][s].view(-1)
        formula = 'ij,'+self.player_dim(other_player)+'->'+self.player_dim(player)
        r_mean[player][second_player][s] = torch.einsum(formula, RM_i[s], strategy).view(-1,1)
    return r_mean
  
  def partial_expected_reward_other(self):
    r_mean = {}
    for player in self.players():
      r_mean[player] = {}
    for s, player in self.state_player_pairs():
        RM_i = self.R[player]
        other_player = self.other_player(player)
        strategy = self.pi[other_player][s].view(-1)
        formula = 'ij,'+self.player_dim(other_player)+'->'+self.player_dim(player)
        r_mean[player][s] = torch.einsum(formula, RM_i[s], strategy).view(-1,1)
    return r_mean
  
  # TODO: consider other 2 cases or transition matrices
  # TODO: consider more than 2 players
  def next_value_matrices(self, state):
    det, dep = self.transition_type[str(state)]
    N_A1 = self.A['1'][str(state)]
    N_A2 = self.A['2'][str(state)]
    vs = torch.zeros((N_A1,N_A2,2)).to(self.device)
    if det and dep: 
      for a1 in range(0,N_A1):
        for a2 in range(0,N_A2):
          _, next_state = self.transition_map(state, [a1,a2])
          vs[a1,a2,:] = self.v[self.get_state_index(next_state),:]
    elif (not det) and (not dep):
      _, transition_dic = self.transition_map(state, [])
      next_v = torch.zeros(1,2).to(self.device)
      for next_state, transition_prob in transition_dic:
        next_v = next_v + self.v[self.get_state_index(next_state),:].view(1,-1) * transition_prob      
      for a1 in range(0,N_A1):
        for a2 in range(0,N_A2):
          vs[a1,a2,:] = next_v.view(-1)
    return vs
  
  # TODO: consider more than 2 players
  def partial_next_values(self):
    # Create next-value dictionary for each player combination
    next_v = {}
    for player in self.players():
      next_v[player] = {'1':{}, '2':{}}

    # Fill dictionary 
    for state in self.S_no_string:
      next_state_value_matrix = self.next_value_matrices(state)
      strategy_1 = self.pi['1'][str(state)].squeeze(1)
      strategy_2 = self.pi['2'][str(state)].squeeze(1)
      
      # Calculate mean next value when considering the strategy of one of the players. Output: array of size m^i(s)
      for player_id in self.players_id():
        next_v['1'][str(player_id+1)][str(state)] = torch.einsum('ij,j->i', next_state_value_matrix[:,:,player_id], strategy_2).view(-1,1)
        next_v['2'][str(player_id+1)][str(state)] = torch.einsum('ij,i->j', next_state_value_matrix[:,:,player_id], strategy_1).view(-1,1)  
    return next_v
  
  def next_value_dictionary(self):
    next_v_dic = {}
    for state in self.S_no_string:
      next_v_dic[str(state)] = self.next_value_matrices(state)
    return next_v_dic
  
  # TODO: consider more than 2 players
  def bellman_projection(self):
    r_mean = self.expected_reward()
    next_v = torch.zeros_like(self.v)
    for s in self.S_no_string:
      next_state_value_matrix = self.next_value_matrices(s)
      strategy_1 = self.pi['1'][str(s)].squeeze(1)
      strategy_2 = self.pi['2'][str(s)].squeeze(1)
      next_value_1 = torch.einsum('ijk,i->jk', next_state_value_matrix, strategy_1)
      next_v[self.get_state_index(s),:] = torch.einsum('jk,j->k', next_value_1, strategy_2)
    return r_mean + beta * next_v

  # TODO: consider more than 2 players
  def bellman_partial_projection(self):
    r_mean = self.partial_expected_reward()
    next_v = self.partial_next_values()

    bellman_projection_dic = {'1':{}, '2':{}}
    for player in self.players():
      bellman_projection_dic[player] = {'1':{}, '2':{}}
    for s, player in self.state_player_pairs():
      for second_player in self.players():
        bellman_projection_dic[player][second_player][s] = (
            r_mean[player][second_player][s] + beta * next_v[player][second_player][s])
    return bellman_projection_dic

  # TODO: consider more than 2 players
  def bellman_partial_projection_other(self):
    r_mean = self.partial_expected_reward_other()
    next_v = {'1':{}, '2':{}}
    for state in self.S_no_string:
      next_state_value_matrix = self.next_value_matrices(state)
      strategy_1 = self.pi['1'][str(state)].squeeze(1)
      strategy_2 = self.pi['2'][str(state)].squeeze(1)
      # mean next value when considering the strategy of the other player. Output: array of size m^i(s)
      next_v['1'][str(state)] = torch.einsum('ij,j->i', next_state_value_matrix[:,:,0], strategy_2).view(-1,1)
      next_v['2'][str(state)] = torch.einsum('ij,i->j', next_state_value_matrix[:,:,1], strategy_1).view(-1,1)
    bellman_projection_dic = {'1':{}, '2':{}}
    for s, player in self.state_player_pairs():
      bellman_projection_dic[player][s] = r_mean[player][s] + beta * next_v[player][s]
    return bellman_projection_dic
  
  #-----------------------------------------------------------------------------
  # Methods related with the associated non-linear optimization problem

  # Calculation of original target function: Bellman approximation error
  def calculate_bellman_error(self):     
    v_estimated = self.bellman_projection()
    f_bellman = (self.v - v_estimated).sum()
    return f_bellman

  def calculate_nash_restrictions(self):
    q_estimated = self.bellman_partial_projection_other() # Dic. with an array of 'q'-values for each agent 
    g_nash = {'1':{}, '2':{}}
    for s, player in self.state_player_pairs():
      g_nash[player][s] = q_estimated[player][s] - self.v[self.get_state_index(s), self.get_player_id(player)]
    return g_nash
  
  def calculate_restriction_vector(self):
    g_nash = self.calculate_nash_restrictions()
    pi_vector = self.pi2vec()
    pi_sum_vector = self.pi_sum()
    g_list = []
    for player in self.players():
      for s in self.S:
        g_list.append(g_nash[player][s])
    for player in self.players():
      g_list.append(-pi_vector[player])
    for player in self.players():
      g_list.append(pi_sum_vector[player]-1)
    g_vector = torch.cat(g_list, dim=0)
    return g_vector
  
  # Methods to calculate gradients
  def bellman_error_gradients(self):
    P = self.transition_matrix()
    bellman_error_grad_v = (torch.eye(self.N_S).to(self.device) - torch.t(P)).sum(1, keepdim=True)

    q_individual = self.bellman_partial_projection()
    bellman_error_grad_pi = {'1':{}, '2':{}}
    for s, player in self.state_player_pairs():
      bellman_error_grad_pi[player][s] = 0.0
      for second_player in self.players():
        bellman_error_grad_pi[player][s] = bellman_error_grad_pi[player][s] - q_individual[player][second_player][s]  
    return bellman_error_grad_v, bellman_error_grad_pi

  def nash_restriction_gradients(self):
    P_partial = self.partial_transition_matrices()
    next_value_dic = self.next_value_dictionary()
    consistent_R = self.player_consistent_reward_matrices()

    nash_restriction_grad_v = {'1':{}, '2':{}}
    nash_restriction_grad_pi = {'1':{}, '2':{}}

    for s, player in self.state_player_pairs():
      Delta = torch.zeros_like(P_partial[s][player])
      Delta[:, self.get_state_index(s)] = 1.0
      nash_restriction_grad_v[player][s] = beta * P_partial[s][player]  - Delta
    
      other_player = self.other_player(player)
      next_value_matrix = next_value_dic[s][:,:,self.get_player_id(other_player)]
      r_matrix = consistent_R[other_player][s]
      if player == '1':
        next_value_matrix = torch.t(next_value_matrix)
      nash_restriction_grad_pi[player][s] = r_matrix + beta * next_value_matrix
    return nash_restriction_grad_v, nash_restriction_grad_pi
  
  # TODO: consider more than 2 players
  def bellman_error_gradient_vector(self):
    grad_f_v, grad_f_pi = self.bellman_error_gradients() 
    grad_f_vector = torch.zeros(self.n_vars, 1).to(self.device)
    
    for i in range(0,2):
      grad_f_vector[i*self.N_S:(i+1)*self.N_S,:] = grad_f_v.clone()
    
    for player in self.players():
      grad_f_pi_list = []
      for s in self.S:
        if self.more_than_one_action(player, s):
          grad_f_pi_list.append(grad_f_pi[player][s].clone())  
      grad_f_pi_vector = torch.cat(grad_f_pi_list, dim=0)
      y0 = 2*self.N_S + self.get_player_id(player) * self.N_A_reduced['1']
      yf = 2*self.N_S + self.N_A_reduced['1'] + self.get_player_id(player) * self.N_A_reduced['2']
      grad_f_vector[y0:yf,:] = grad_f_pi_vector

    return grad_f_vector

  def nash_restrictions_pi_jacobian_matrix(self, player, grad_g_pi):
    other_player = self.other_player(player)
    grad_g_pi_list = []
    s_not_added_list = []
    for s in self.S:
      if self.more_than_one_action(player, s):
        grad = grad_g_pi[player][s].clone()
        if len(s_not_added_list) == 0:
          grad_g_pi_list.append(grad)
        else:
          n_null_rows = 0
          for s_ in s_not_added_list:
            n_null_rows += self.A[other_player][s_]
          n_rows = n_null_rows + grad.shape[0]
          n_cols = grad.shape[1]
          null_rows_and_grad_g_pi = torch.zeros(n_rows, n_cols).to(self.device)
          null_rows_and_grad_g_pi[-n_rows+n_null_rows:,:] = grad
          grad_g_pi_list.append(null_rows_and_grad_g_pi)
          s_not_added_list = []
      else:
        s_not_added_list.append(s)
    if len(s_not_added_list) != 0:
      n_null_rows = 0
      for s_ in s_not_added_list:
        n_null_rows += self.A[other_player][s_]
      n_rows = n_null_rows + grad_g_pi_list[-1].shape[0]
      n_cols = grad_g_pi_list[-1].shape[1]
      null_rows_and_grad_g_pi = torch.zeros(n_rows, n_cols).to(self.device)
      null_rows_and_grad_g_pi[:n_rows-n_null_rows,:] = grad_g_pi_list[-1].clone()
      grad_g_pi_list[-1] = null_rows_and_grad_g_pi
    
    J_gmi_pit = torch.block_diag(*grad_g_pi_list)
    return J_gmi_pit

  def build_grad_tensors(self):           
    grad_f_vector = self.bellman_error_gradient_vector()
    grad_g_v, grad_g_pi = self.nash_restriction_gradients()
    grad_g_matrix = torch.zeros(self.n_restrictions, self.n_vars).to(self.device)
    
    for player in self.players():
      other_player = self.other_player(player)

      # fill jacobian with restriction gradients for v
      grad_g_v_list = [grad_g_v[player][s].clone() for s in self.S]
      J_gi_vi = torch.cat(grad_g_v_list, dim=0)
      y0 = self.get_player_id(player) * self.N_A_total['1']
      yf = self.N_A_total['1'] + self.get_player_id(player) * self.N_A_total['2']
      x0 = self.get_player_id(player) * self.N_S
      xf = x0 + self.N_S
      grad_g_matrix[y0:yf,x0:xf] = J_gi_vi

      # fill jacobian with nash restriction gradients for pi 
      J_gmi_pit = self.nash_restrictions_pi_jacobian_matrix(player, grad_g_pi)
      y0 = self.get_player_id(other_player) * self.N_A_total['1']
      yf = self.N_A_total['1'] + self.get_player_id(other_player) * self.N_A_total['2']
      x0 = 2*self.N_S + self.get_player_id(player) * self.N_A_reduced['1']
      xf = 2*self.N_S + self.N_A_reduced['1'] + self.get_player_id(player) * self.N_A_reduced['2']
      grad_g_matrix[y0:yf,x0:xf] = J_gmi_pit

      # fill jacobian with positivity restriction gradients for pi 
      J_gpi_plus = -torch.eye(self.N_A_reduced[player]).to(self.device)
      y0 = self.N_A_total['1'] + self.N_A_total['2'] + self.get_player_id(player) * self.N_A_reduced['1']
      yf = (self.N_A_total['1'] + self.N_A_total['2'] + self.N_A_reduced['1'] 
            + self.get_player_id(player) * self.N_A_reduced['2'])
      grad_g_matrix[y0:yf,x0:xf] = J_gpi_plus

      # fill jacobian with unitary sum restriction gradients for pi
      grad_gpi_one_list = []
      for s in self.S:
        NA = self.A[player][s]
        if self.more_than_one_action(player, s):
          grad_gpi_one_list.append(torch.ones(1,NA).to(self.device))

      J_gpi_one = torch.block_diag(*grad_gpi_one_list)
      y0 = (self.N_A_total['1'] + self.N_A_total['2'] + self.N_A_reduced['1'] 
            + self.N_A_reduced['2'] + self.get_player_id(player) * self.N_S_reduced['1'])
      yf = (self.N_A_total['1'] + self.N_A_total['2'] + self.N_A_reduced['1'] + self.N_A_reduced['2'] 
            + self.N_S_reduced['1'] + self.get_player_id(player) * self.N_S_reduced['2'])
      grad_g_matrix[y0:yf,x0:xf] = J_gpi_one

    return grad_f_vector, grad_g_matrix

## Prisoner's Dilemma With Incentives

Now, we will create an instance of a stochastic game that corresponds to the prisoner's dilemma game. This game will include a modification that allows for mutual incentives between the prisoners.

In [3]:
# Define player set N
N = 2

# Define state set S
M = 5
G = [('G',)]
O = [('O',i) for i in range(1,2+1)]
E1 = [('E1',i,j) for i in range(0,2) for j in range(0,M+1)]
E2 = [('E2',i,j) for i in range(0,2) for j in range(0,M+1)]
R1 = [('R1',i) for i in range(0,2)]
R2 = [('R2',i) for i in range(0,2)]
S = list(itertools.chain(G.copy(), O.copy(), E1.copy(), E2.copy(), R1.copy(), R2.copy())) 

# Define action set A^i(s) per player and state
A = {'1':{}, '2':{}}
for player_id in range(0, N):
  player = str(player_id+1)
  other_player_id = 1-player_id
  other_player = str(other_player_id+1)
  A[player][str(('G',))] = 2
  A[player][str(('O',player_id+1))] = 2*(M+1)
  A[player][str(('O',other_player_id+1))] = 1
  for i in range(0,2):
    for j in range(0,M+1):
      A[player][str(('E'+player,i,j))] = 2
      A[player][str(('E'+other_player,i,j))] = 1
    A[player][str(('R'+player,i))] = 2
    A[player][str(('R'+other_player,i))] = 1

# Define discount factor
beta = 0.99

In [4]:
# Define payoff map R
R1 = np.array([[3.,0.],[5.,1.]])
R2 = np.array([[3.,5.],[0.,1.]])

def offer_accepted(action):
  if action == 0:
    return True
  else:
    return False

def prisoner_dilemma_rewards(state, actions):
  if 'G' in state:
    r1 = R1[actions[0], actions[1]]
    r2 = R2[actions[0], actions[1]]
  elif 'O' in state:
    r1 = r2 = 0.0
  elif 'E1' in state:
    _, _, offeral = state
    if offer_accepted(actions[0]):
      r1 = offeral
      r2 = -offeral
    else:
      r1 = r2 = 0
  elif 'E2' in state:
    _, _, offeral = state
    if offer_accepted(actions[1]):
      r1 = -offeral
      r2 = offeral
    else:
      r1 = r2 = 0
  elif 'R1' in state:
    _, i = state
    r1 = R1[actions[0], i]
    r2 = R2[actions[0], i]
  elif 'R2' in state:
    _, i = state
    r1 = R1[i, actions[1]]
    r2 = R2[i, actions[1]]
  else:
    assert 0 == 1, 'Invalid state'
  return r1, r2

def prisoner_dilemma_reward_matrices(s):
  if 'G' in s:
    RM1 = R1.copy()
    RM2 = R2.copy()
  else:
    N_A1 = A['1'][str(s)]
    N_A2 = A['2'][str(s)]
    RM1 = np.zeros((N_A1,N_A2))
    RM2 = np.zeros((N_A1,N_A2))
    for a1 in range(0,N_A1):
      for a2 in range(0,N_A2):
        r1, r2 = prisoner_dilemma_rewards(s, [a1,a2])
        RM1[a1,a2] = r1
        RM2[a1,a2] = r2
  return RM1, RM2  

R = {'1':{}, '2':{}}
for s in S:
  RM1, RM2 = prisoner_dilemma_reward_matrices(s)
  R['1'][str(s)] = torch.FloatTensor(RM1).clone()
  R['2'][str(s)] = torch.FloatTensor(RM2).clone()

In [5]:
# Define transition map
def prisoner_dilemma_transition_info(state, actions):
  if 'G' in state or 'R1' in state or 'R2' in state:
    T = [(('O',1), 0.5), (('O',2), 0.5)]
    return ('R', T)
  elif 'O' in state:
    player_id = state[1]-1
    other_player_id = 1 - player_id
    other_player = str(other_player_id + 1)
    offeral = actions[player_id] // 2
    action_requested = actions[player_id] % 2
    return ('D', ('E'+other_player, action_requested, offeral))
  elif 'E1' in state:
    _, action_requested, _ = state
    if offer_accepted(actions[0]):
      return ('D', ('R2', action_requested))
    else:
      return ('D', ('G',))
  elif 'E2' in state:
    _, action_requested, _ = state
    if offer_accepted(actions[1]):
      return ('D', ('R1', action_requested))
    else:
      return ('D', ('G',))   
  else:
    assert 0 == 1, 'Invalid state' 

prisoner_dilemma_transition_types = {}
for state in S:
  if 'G' in state or 'R1' in state or 'R2' in state:
    prisoner_dilemma_transition_types[str(state)] = (0,0) 
  else:
    prisoner_dilemma_transition_types[str(state)] = (1,1) # entries correspond to deterministic behaviour and dependence on actions

T = (prisoner_dilemma_transition_info, prisoner_dilemma_transition_types)

In [24]:
game = multi_player_game(N, S, A, R, T, beta, device).to(device)

In [25]:
gv, gm = game.build_grad_tensors()

# Game Solver

Still missing...

In [ ]:
class game_solver():
  def __init__(alpha, gamma_0, eta, nu, rho_0, r_value, c_0)
    # Store solver hyperparameters
    self.alpha = alpha
    self.gamma_0 = gamma_0
    self.eta = eta
    self.nu = nu
    self.rho_0 = rho_0
    self.r_value = r_value
    self.c_0 = c_0
  




  

  def optimize_game(self, game, n_epochs=100, verbose=False):
    game_new = game.copy()
    pi_0, v_0 = game_0()
    f_0 = calculate_bellman_error(pi_0, v_0)
    c_vector = c*torch.ones(n_restrictions,1).to('cuda')
    c_vector = mask_equality_restrictions(c_vector)
    with torch.no_grad():
      for epoch in range(0, n_epochs):
        pi, v = game_new()
        pi_vector = game_new.pi2vec()
        pi_sum_vector = game_new.pi_sum()
        f = calculate_bellman_error(pi, v)
        g_nash = calculate_nash_restrictions(pi, v)
        g_vector = g_dics2vec(g_nash, pi_vector, pi_sum_vector)
        grad_f_v, grad_f_pi = bellman_error_gradients(pi, v)
        
        grad_g_v, grad_g_pi = nash_restriction_gradients(pi, v)
        
        grad_f_vector, grad_g_matrix = build_grad_tensors(grad_f_v, grad_f_pi, grad_g_v, grad_g_pi)
        d0_vector, norm_2_d0, duals_0_vector, A_matrix, b_vector = calculate_descent_direction(g_vector, grad_f_vector, grad_g_matrix)
        c_vector = update_c_vector(c_vector, duals_0_vector)

        d_vector, duals_vector, rho = calculate_feasible_direction(g_vector, c_vector, duals_0_vector, 
                                                                  norm_2_d0, A_matrix, b_vector, rho, 
                                                                  grad_f_vector, grad_g_matrix)
        d_v, d_pi, duals = vec2dic(d_vector, duals_vector)

        g_nash_satisfied, max_g_nash, prod_zero_satisfied, max_prod_zero = check_nash_KKT_conditions(pi, v, duals)
        pi_KKT = check_pi_KKT_conditions(pi_vector, pi_sum_vector, duals)
        g_pi_plus_satisfied, max_g_pi_plus, prod_zero_plus_satisfied, max_prod_zero_plus = pi_KKT[:4] 
        g_pi_one_satisfied, max_g_pi_one, prod_zero_one_satisfied, max_prod_zero_one = pi_KKT[4:]

        game_temp, found_feasible_step_size, n_step, f_new = feasible_gradient_descent(game_new, f, g_vector, c_vector, d_vector, 
                                                                                      grad_f_vector, grad_g_matrix, d_v, d_pi, 
                                                                                      duals_vector, verbose=verbose) # TODO: check if passing wrong grad_f
        if found_feasible_step_size:
          game_new = game_temp
          delta_f = (f_new - f_0) / f_0 * 100
          print('Epoch: {}, f: {:.5e}, delta f: {:.5e}%, nash satisfied: {}, max g: {:.3e}, dual nash satisfied: {}, max product: {:.3e}, rho: {:.3e}, norm2 d0:{:.3e}'.format(
              epoch, f_new.item(), delta_f.item(), g_nash_satisfied, max_g_nash, prod_zero_satisfied, max_prod_zero, rho, norm_2_d0))
          
          print('Epoch: {}, >0 satisfied: {}, max g>0: {:.3e}, dual >0 satisfied: {}, max product >0: {:.3e}, =1 satisfied: {}, max g=1: {:.3e}, dual =1 satisfied: {}, max product =1: {:.3e}'.format(
            epoch, g_pi_plus_satisfied, max_g_pi_plus, prod_zero_plus_satisfied, max_prod_zero_plus, g_pi_one_satisfied, max_g_pi_one, prod_zero_one_satisfied, max_prod_zero_one))
        else:
          break
          
    return game_new, rho     

In [ ]:
def mask_inequality_restrictions(vec):
  masked_vector = vec.clone()
  masked_vector[-N_S_reduced['1']-N_S_reduced['2']:,:] = 0.0
  return masked_vector

def mask_equality_restrictions(vec):
  masked_vector = vec.clone()
  masked_vector[:-N_S_reduced['1']-N_S_reduced['2'],:] = 0.0
  return masked_vector






In [ ]:
def reward_baselines(RM, pi):
  r_mean = expected_reward(RM, pi)
  r_baseline = r_mean.mean(0).view(-1,1).detach().cpu().numpy()
  return r_baseline

In [ ]:
def cost_vector_fixed_policies(pi):
  P = transition_matrix(pi)
  cost_vector = (1 - beta * P.sum(0)).view(-1,1).detach().cpu().numpy() / N_S
  return cost_vector

def restriction_matrices_fixed_policies(pi):
  transition_matrices = partial_transition_matrices(pi)
  restriction_matrices = {'1':[], '2':[]}
  for s, player_id in state_player_pairs():
    temp_matrix = - beta * transition_matrices[str(s)][str(player_id)]
    temp_matrix[:, get_state_index(s)] = temp_matrix[:, get_state_index(s)] + 1
    restriction_matrices[str(player_id)].append(temp_matrix)
  for player in players_str():
    restriction_matrices[player] = -torch.cat(restriction_matrices[player], dim=0).detach().cpu().numpy()
  return restriction_matrices

def restriction_vectors_fixed_policies(RM, pi, alpha=0.1):
  r_mean = partial_expected_reward_other(RM, pi)
  restriction_vectors = {'1':[], '2':[]}
  for s, player in state_player_str_pairs():
    restriction_vectors[player].append(r_mean[player][s].view(-1,1))
  for player in players_str():
    restriction_vectors[player] = -(torch.cat(restriction_vectors[player], dim=0)+alpha).detach().cpu().numpy()
  return restriction_vectors


def parameters_fixed_policies(game_, alpha):
  pi = game_()[0]
  c = cost_vector_fixed_policies(pi)
  f0 = reward_baselines(RM, pi)
  A_ub = restriction_matrices_fixed_policies(pi)
  b_ub = restriction_vectors_fixed_policies(RM, pi, alpha)
  return c, f0, A_ub, b_ub


def calculate_initial_v(game_, alpha=10):
  v0 = np.zeros((N_S,2))
  c, f0, A_ub, b_ub = parameters_fixed_policies(game_, alpha=alpha)
  for player_id in players():
    temp_res = linprog(c, A_ub=A_ub[str(player_id)], b_ub=b_ub[str(player_id)])
    v0[:,player_id-1] = temp_res.x
  return v0

In [ ]:





def check_nash_KKT_conditions(pi, v, lambda_nash, tol=1e-8):
  with torch.no_grad():
    # Calculate restrictions
    g_nash = calculate_nash_restrictions(pi, v)

    g_nash_satisfied = True
    product_zero_satisfied = True
    
    max_g_nash = -np.infty
    max_product_zero = -np.infty
    
    for player in players_str():
      remaining_duals = lambda_nash[player].clone()
      for s in S_str:
        if more_than_one_action_in_s(player, s):
          NA = N_A_S[s][player]
          g_nash_satisfied = g_nash_satisfied and torch.all(g_nash[player][s] <= 0)
          max_g_nash = max(max_g_nash, g_nash[player][s].max().item())

          lambda_g_nash_product = g_nash[player][s].view(-1) * remaining_duals[:NA,:].view(-1)
          product_zero_satisfied = product_zero_satisfied and torch.all(lambda_g_nash_product.abs() <= tol)
          max_product_zero = max(max_product_zero, lambda_g_nash_product.abs().max().item())
          remaining_duals = remaining_duals[NA:,:]
    return g_nash_satisfied, max_g_nash, product_zero_satisfied, max_product_zero


def check_pi_KKT_conditions(pi_vector, pi_sum_vector, duals, tol=1e-8):
  with torch.no_grad():
    g_pi_plus_satisfied = True
    g_pi_one_satisfied = True

    product_zero_plus_satisfied = True
    product_zero_one_satisfied = True

    max_g_pi_plus = -np.infty
    max_g_pi_one = -np.infty

    max_product_zero_plus = -np.infty
    max_product_zero_one = -np.infty

    for player in players_str():
      g_pi_plus_satisfied = g_pi_plus_satisfied and torch.all(pi_vector[player] >= 0)
      g_pi_one_satisfied = g_pi_one_satisfied and torch.all(pi_sum_vector[player] <= 1)

      max_g_pi_plus = max(max_g_pi_plus, -pi_vector[player].min().item())
      max_g_pi_one = max(max_g_pi_one, (pi_sum_vector[player]-1).min().item())

      NAt = N_A_total[player]
      NAr = N_A_reduced[player]
      lambda_g_pi_plus_product = -pi_vector[player].view(-1) * duals[player][NAt:NAt+NAr,:].view(-1) # TODO: fix for assymetric number of actions
      product_zero_plus_satisfied = product_zero_plus_satisfied and torch.all(lambda_g_pi_plus_product.abs() <= tol)
      max_product_zero_plus = max(max_product_zero_plus, lambda_g_pi_plus_product.abs().max().item())
      lambda_g_pi_one_product = (pi_sum_vector[player].view(-1)-1) * duals[player][NAt+NAr:,:].view(-1)
      product_zero_one_satisfied = product_zero_one_satisfied and torch.all(lambda_g_pi_one_product.abs() <= tol)
      max_product_zero_one = max(max_product_zero_one, lambda_g_pi_one_product.abs().max().item())
   
    return (g_pi_plus_satisfied, max_g_pi_plus, product_zero_plus_satisfied, max_product_zero_plus, 
            g_pi_one_satisfied, max_g_pi_one, product_zero_one_satisfied, max_product_zero_one)

In [ ]:





def calculate_descent_direction(g_vector, grad_f_vector, grad_g_matrix):
  g_vector_ineq = mask_inequality_restrictions(g_vector)
  g_vector_eq = mask_equality_restrictions(g_vector)
  g_diag_matrix = torch.diag(r_value * g_vector_ineq.view(-1))

  A_matrix = g_diag_matrix - torch.einsum('ik,jk->ij', grad_g_matrix, grad_g_matrix) 
  b_vector = torch.einsum('ij,jk->ik', grad_g_matrix, grad_f_vector) - g_vector_eq 
  duals_0_vector = torch.solve(b_vector, A_matrix)[0]

  d0_vector = - grad_f_vector - torch.einsum('ij,ik->jk', grad_g_matrix, duals_0_vector)
  norm_2_d0 = d0_vector.pow(2).sum().item() 
  return d0_vector, norm_2_d0, duals_0_vector, A_matrix, b_vector


def calculate_feasible_direction(g_vector, c_vector, duals_0_vector, 
                                 norm_2_d0, A_matrix, b_vector, rho,
                                  grad_f_vector, grad_g_matrix):
  b_unitary = torch.solve(torch.ones_like(b_vector), A_matrix)[0]
  g_vector_eq = mask_equality_restrictions(g_vector)
  dot = (g_vector_eq.view(-1) * b_unitary.view(-1)).sum()

  div = (duals_0_vector.sum() + c_vector.sum() + dot).item()
  if div > 0:
      rho_1 = (1-alpha) / div
      if rho_1 < rho:
        rho = 0.5 * rho_1

  duals_vector = torch.solve(b_vector - rho * norm_2_d0, A_matrix)[0]
  d_vector = - grad_f_vector - torch.einsum('ij,ik->jk', grad_g_matrix, duals_vector)
  return d_vector, duals_vector, rho


def calculate_auxiliary_bellman_error(f, g_vector, c_vector):
  return (f - (c_vector.view(-1) * g_vector.view(-1)).sum()).item()


def feasible_gradient_descent(game_0, f, g_vector, c_vector, d_vector,
                              grad_f_vector, grad_g_matrix, d_v, d_pi, 
                              duals_vector, max_steps=1200, 
                              verbose=False):
  # Calculate loss function
  theta = calculate_auxiliary_bellman_error(f, g_vector, c_vector)
  
  # Calculate required decrement in the loss function
  grad_theta_vector = grad_f_vector + torch.einsum('ij,ik->jk', grad_g_matrix, c_vector)
  decrement = (d_vector.view(-1) * grad_theta_vector.view(-1)).sum().item()

  # Procedure to find feasible step size 
  found_feasible_step_size = False
  step_size = 1.0
  n_step = 0
  max_steps = max(1, max_steps)
  while (not found_feasible_step_size) and (n_step < max_steps):
    game_temp = copy_game(game_0)
    # Update parameters performing step in feasible descent direction 
    game_temp.v.data.add_(step_size * d_v)
    for s in S_str:
      if more_than_one_action_in_s(player='1', s=s):
        game_temp.pi1[s].data.add_(step_size * d_pi['1'][s])
      if more_than_one_action_in_s(player='2', s=s):
        game_temp.pi2[s].data.add_(step_size * d_pi['2'][s])

    pi_temp, v_temp = game_temp()    
    pi_vector_temp = game_temp.pi2vec()
    pi_sum_vector_temp = game_temp.pi_sum()
    f_temp = calculate_bellman_error(pi_temp, v_temp)
    g_nash_temp = calculate_nash_restrictions(pi_temp, v_temp)
    g_vector_temp = g_dics2vec(g_nash_temp, pi_vector_temp, pi_sum_vector_temp)
    theta_temp = calculate_auxiliary_bellman_error(f_temp, g_vector_temp, c_vector)

    gamma = gamma_0 * torch.ones_like(duals_vector)
    gamma[duals_vector < 0] = 1.0
    gamma[-N_S_reduced['1']-N_S_reduced['2']:,:] = 0.0

    theta_decreased = theta_temp <= theta + eta * step_size * decrement
    g_v_valid = torch.all(g_vector_temp[:114] <= (g_vector * gamma)[:114]).item()
    g_pi_valid = torch.all(g_vector_temp[114:] <= (g_vector * gamma)[114:]).item()
    g_valid = g_v_valid and g_pi_valid
    
    # Check if the current step size results in a feasible descent direction
    if theta_decreased and g_valid:
      found_feasible_step_size = True
      
    step_size /= nu
    n_step += 1
    if verbose:
      print(theta_decreased, g_v_valid, g_pi_valid)
      print("Step: {}, Step size: {:.3e}, Found feasible: {}".format(
          n_step, step_size, found_feasible_step_size))
  return game_temp, found_feasible_step_size, n_step, f_temp


def vec2dic(d_vector, duals_vector):
  d_v = torch.zeros((N_S,2)).to('cuda')  
  d_pi = {'1':{}, '2':{}}
  duals = {}

  # Store gradients related to v in the proper shape
  for i in range(0,2):
    d_v[:,i] = d_vector[i*N_S:(i+1)*N_S,:].view(-1)
  
  # Store gradients related to pi in dictionaries for each player and state
  for player in players_str():
    y0 = 2*N_S + get_player_id(player) * N_A_reduced['1']
    yf = 2*N_S + N_A_reduced['1']  + get_player_id(player) * N_A_reduced['2'] 
    d_pi_vector = d_vector[y0:yf,:]
    n = 0
    for s in S_str:      
      if more_than_one_action_in_s(player, s):
        NA = N_A_S[s][player]
        d_pi[player][s] = d_pi_vector[n:n+NA,:]
        n = n + NA
      else:
        d_pi[player][s] = None      

  # Separate duals for each player  
  for player in players_str():
    i = get_player_id(player)
    other_player_ = other_player(player)
    n_restrictions_player = N_A_total[player] + N_A_reduced[player] + N_S_reduced[player]
    duals[player] = torch.zeros(n_restrictions_player,1).to('cuda')
    
    y0 = get_player_id(player) * N_A_total['1']
    yf = N_A_total['1']  + get_player_id(player) * N_A_total['2'] 
    duals[player][:N_A_total[player],:] = duals_vector[y0:yf,:]
    
    y0 = N_A_total['1'] + N_A_total['2'] + get_player_id(player) * N_A_reduced['1']
    yf = N_A_total['1'] + N_A_total['2'] + N_A_reduced['1'] + get_player_id(player) * N_A_reduced['2'] 
    duals[player][N_A_total[player]:N_A_total[player]+N_A_reduced[player],:] = duals_vector[y0:yf,:]

    y0 = (N_A_total['1'] + N_A_total['2'] + N_A_reduced['1'] + N_A_reduced['2'] 
          + get_player_id(player) * N_S_reduced['1'])
    yf = (N_A_total['1'] + N_A_total['2'] + N_A_reduced['1'] + N_A_reduced['2'] 
          + N_S_reduced['1'] + get_player_id(player) * N_S_reduced['2']) 
    duals[player][-N_S_reduced[player]:,:] = duals_vector[y0:yf,:]

  return d_v, d_pi, duals


def update_c_vector(c_vector, duals_0_vector):
  new_c_vector = c_vector.clone()
  entries_to_update = c_vector < -1.2*duals_0_vector
  new_c_vector[entries_to_update] = -2*duals_0_vector[entries_to_update]
  new_c_vector = mask_equality_restrictions(new_c_vector)
  return new_c_vector


def optimize_game(game_0, rho, n_epochs=100, verbose=False):
  game_new = copy_game(game_0)
  pi_0, v_0 = game_0()
  f_0 = calculate_bellman_error(pi_0, v_0)
  c_vector = c*torch.ones(n_restrictions,1).to('cuda')
  c_vector = mask_equality_restrictions(c_vector)
  with torch.no_grad():
    for epoch in range(0, n_epochs):
      pi, v = game_new()
      pi_vector = game_new.pi2vec()
      pi_sum_vector = game_new.pi_sum()
      f = calculate_bellman_error(pi, v)
      g_nash = calculate_nash_restrictions(pi, v)
      g_vector = g_dics2vec(g_nash, pi_vector, pi_sum_vector)
      grad_f_v, grad_f_pi = bellman_error_gradients(pi, v)
      grad_g_v, grad_g_pi = nash_restriction_gradients(pi, v)
      
      grad_f_vector, grad_g_matrix = build_grad_tensors(grad_f_v, grad_f_pi, grad_g_v, grad_g_pi)
      d0_vector, norm_2_d0, duals_0_vector, A_matrix, b_vector = calculate_descent_direction(g_vector, grad_f_vector, grad_g_matrix)
      c_vector = update_c_vector(c_vector, duals_0_vector)

      d_vector, duals_vector, rho = calculate_feasible_direction(g_vector, c_vector, duals_0_vector, 
                                                                 norm_2_d0, A_matrix, b_vector, rho, 
                                                                 grad_f_vector, grad_g_matrix)
      d_v, d_pi, duals = vec2dic(d_vector, duals_vector)

      g_nash_satisfied, max_g_nash, prod_zero_satisfied, max_prod_zero = check_nash_KKT_conditions(pi, v, duals)
      pi_KKT = check_pi_KKT_conditions(pi_vector, pi_sum_vector, duals)
      g_pi_plus_satisfied, max_g_pi_plus, prod_zero_plus_satisfied, max_prod_zero_plus = pi_KKT[:4] 
      g_pi_one_satisfied, max_g_pi_one, prod_zero_one_satisfied, max_prod_zero_one = pi_KKT[4:]

      game_temp, found_feasible_step_size, n_step, f_new = feasible_gradient_descent(game_new, f, g_vector, c_vector, d_vector, 
                                                                                     grad_f_vector, grad_g_matrix, d_v, d_pi, 
                                                                                     duals_vector, verbose=verbose) # TODO: check if passing wrong grad_f
      if found_feasible_step_size:
        game_new = game_temp
        delta_f = (f_new - f_0) / f_0 * 100
        print('Epoch: {}, f: {:.5e}, delta f: {:.5e}%, nash satisfied: {}, max g: {:.3e}, dual nash satisfied: {}, max product: {:.3e}, rho: {:.3e}, norm2 d0:{:.3e}'.format(
            epoch, f_new.item(), delta_f.item(), g_nash_satisfied, max_g_nash, prod_zero_satisfied, max_prod_zero, rho, norm_2_d0))
        
        print('Epoch: {}, >0 satisfied: {}, max g>0: {:.3e}, dual >0 satisfied: {}, max product >0: {:.3e}, =1 satisfied: {}, max g=1: {:.3e}, dual =1 satisfied: {}, max product =1: {:.3e}'.format(
          epoch, g_pi_plus_satisfied, max_g_pi_plus, prod_zero_plus_satisfied, max_prod_zero_plus, g_pi_one_satisfied, max_g_pi_one, prod_zero_one_satisfied, max_prod_zero_one))
      else:
        break
        
  return game_new, rho     

#**Optimization**

In [ ]:
game_b = full_game().to('cuda')

v0 = calculate_initial_v(game_b, alpha=1e-2)
v0 = torch.FloatTensor(v0).to('cuda')
with torch.no_grad():
  game_b.v.data.add_(v0)

In [ ]:
game_b.load_state_dict(torch.load('./game_b35_v2.pth'))

<All keys matched successfully>

In [ ]:
alpha = 0.5
gamma_0 = 0.5
eta = 0.01
nu = 1/0.94
rho_0 = 1.0
rho = rho_0
r_value = 1.0
c = 1e-2

In [ ]:
alpha = 0.1
gamma_0 = 0.1
eta = 0.5
nu = 1/0.95
rho_0 = 1.0
rho = rho_0
r_value = 1.0
c = 1

In [ ]:
game_b5, rho = optimize_game(game_b5, rho, n_epochs=100, verbose=False)

Epoch: 0, f: 2.53130e+01, delta f: -2.03446e-04%, nash satisfied: True, max g: -1.621e-05, dual nash satisfied: False, max product: 4.255e+02, rho: 3.434e-05, norm2 d0:6.010e+01
Epoch: 0, >0 satisfied: True, max g>0: -4.540e-06, dual >0 satisfied: False, max product >0: 1.635e+00, =1 satisfied: True, max g=1: -5.981e-02, dual =1 satisfied: False, max product =1: 6.114e+01
Epoch: 1, f: 2.53129e+01, delta f: -6.10337e-04%, nash satisfied: True, max g: -1.717e-05, dual nash satisfied: False, max product: 4.254e+02, rho: 3.434e-05, norm2 d0:6.045e+01
Epoch: 1, >0 satisfied: True, max g>0: -4.535e-06, dual >0 satisfied: False, max product >0: 1.635e+00, =1 satisfied: True, max g=1: -5.981e-02, dual =1 satisfied: False, max product =1: 6.113e+01
Epoch: 2, f: 2.53129e+01, delta f: -9.11738e-04%, nash satisfied: True, max g: -1.621e-05, dual nash satisfied: False, max product: 4.169e+02, rho: 3.434e-05, norm2 d0:5.939e+01
Epoch: 2, >0 satisfied: True, max g>0: -4.530e-06, dual >0 satisfied: Fa

KeyboardInterrupt: ignored

In [ ]:
game_b3, rho = optimize_game(game_b4, rho, n_epochs=100, verbose=False)

Epoch: 0, f: 1.91609e+00, delta f: -3.18530e-03%, nash satisfied: True, max g: -2.003e-03, dual nash satisfied: False, max product: 5.036e+00, rho: 2.173e-04, norm2 d0:1.130e-02
Epoch: 0, >0 satisfied: True, max g>0: -2.841e-07, dual >0 satisfied: False, max product >0: 1.086e-02, =1 satisfied: True, max g=1: -1.221e-04, dual =1 satisfied: False, max product =1: 8.709e-03
Epoch: 1, f: 1.91607e+00, delta f: -4.57886e-03%, nash satisfied: True, max g: -2.003e-03, dual nash satisfied: False, max product: 5.037e+00, rho: 2.173e-04, norm2 d0:1.123e-02
Epoch: 1, >0 satisfied: True, max g>0: -2.841e-07, dual >0 satisfied: False, max product >0: 1.071e-02, =1 satisfied: True, max g=1: -1.221e-04, dual =1 satisfied: False, max product =1: 8.685e-03
Epoch: 2, f: 1.91604e+00, delta f: -5.77335e-03%, nash satisfied: True, max g: -2.003e-03, dual nash satisfied: False, max product: 5.037e+00, rho: 2.173e-04, norm2 d0:1.116e-02
Epoch: 2, >0 satisfied: True, max g>0: -2.851e-07, dual >0 satisfied: Fa

KeyboardInterrupt: ignored

In [ ]:
game_b3 = full_game().to('cuda')

v3 = calculate_initial_v(game_b3, alpha=0.01)
v3 = torch.FloatTensor(v3).to('cuda')
with torch.no_grad():
  game_b3.v.data.add_(v3)

In [ ]:
game_b3, rho = optimize_game(game_b2, rho, n_epochs=100, verbose=False)

In [ ]:
alpha = 0.5
gamma_0 = 0.3
eta = 0.1
nu = 1/0.98
rho_0 = 1.0
rho = rho_0
r_value = 1.0
c = 1.0

In [ ]:
game_b6 = full_game().to('cuda')
game_b6.load_state_dict(torch.load('./game_b37_v2.pth'))

<All keys matched successfully>

In [ ]:
torch.save(game_b6.state_dict(), './game_b37_v2.pth')

In [ ]:
game_b4 = copy_game(game_b6)

In [ ]:
v0_4 = calculate_initial_v(game_b4, alpha=0.01)
v0_4 = torch.FloatTensor(v0_4).to('cuda')

pi_4, v_4 = game_b4()
f0_4 = calculate_bellman_error(pi_4, v0_4)
f_4 = calculate_bellman_error(pi_4, v_4)

In [ ]:
print(f_4.item(), f0_4.item())

2.3759002685546875 2.6063461303710938


In [ ]:
for s in S_str:
  print(s)
  print(pi_4['1'][s].detach().cpu().numpy())

('G',)
[[0.00515386]
 [0.99474835]]
('O', 1)
[[4.6921283e-01]
 [1.1681265e-03]
 [4.1533099e-03]
 [7.3859107e-04]
 [1.8238504e-01]
 [4.9532688e-04]
 [3.3738485e-01]
 [3.4022375e-04]
 [2.5026791e-03]
 [2.3170366e-04]
 [1.1871576e-03]
 [1.5736901e-04]]
('O', 2)
[[1.]]
('E1', 0, 0)
[[0.33611977]
 [0.6637566 ]]
('E1', 0, 1)
[[0.30288228]
 [0.6969746 ]]
('E1', 0, 2)
[[0.69115424]
 [0.3087085 ]]
('E1', 0, 3)
[[9.991928e-01]
 [6.823313e-04]]
('E1', 0, 4)
[[9.994120e-01]
 [4.648149e-04]]
('E1', 0, 5)
[[9.9960536e-01]
 [2.6499477e-04]]
('E1', 1, 0)
[[0.5356648 ]
 [0.46420684]]
('E1', 1, 1)
[[0.99829036]
 [0.00160284]]
('E1', 1, 2)
[[9.990798e-01]
 [8.028709e-04]]
('E1', 1, 3)
[[9.9943733e-01]
 [4.5240592e-04]]
('E1', 1, 4)
[[9.9962217e-01]
 [2.6901829e-04]]
('E1', 1, 5)
[[9.9972802e-01]
 [1.5317732e-04]]
('E2', 0, 0)
[[1.]]
('E2', 0, 1)
[[1.]]
('E2', 0, 2)
[[1.]]
('E2', 0, 3)
[[1.]]
('E2', 0, 4)
[[1.]]
('E2', 0, 5)
[[1.]]
('E2', 1, 0)
[[1.]]
('E2', 1, 1)
[[1.]]
('E2', 1, 2)
[[1.]]
('E2', 1, 3)
[

In [ ]:
delta_ = 0.0001
game_b5 = copy_game(game_b4)
with torch.no_grad():
  for s in S_str:
    if more_than_one_action_in_s('1',s):
      p = game_b5.pi1[s].clone() + 1e-3
      p[p == p.max()] -= 1e-3
      p *= 0.9997/p.sum()
      print(p)
      # p = 4*(1-delta_)*p**3 + (-6*(1-delta_) + 2*delta_)*p**2 + (3*(1-delta_) - 2*delta_)*p + delta_
      # p *= 0.9997/p.sum()
      nn.init.zeros_(game_b5.pi1[s])
      game_b5.pi1[s].data.add_(p) 
    if more_than_one_action_in_s('2',s):
      p = game_b5.pi2[s].clone() + 1e-3
      p[p == p.max()] -= 1e-3
      p *= 0.9997/p.sum()
      # p = 4*(1-delta_)*p**3 + (-6*(1-delta_) + 2*delta_)*p**2 + (3*(1-delta_) - 2*delta_)*p + delta_
      # p *= 0.9997/p.sum()
      nn.init.zeros_(game_b5.pi2[s])
      game_b5.pi2[s].data.add_(p) 

In [ ]:
v0_5 = calculate_initial_v(game_b5, alpha=0.02)
v0_5 = torch.FloatTensor(v0_5).to('cuda')

pi_5, v_5 = game_b5()
f0_5 = calculate_bellman_error(pi_5, v0_5)
f_5 = calculate_bellman_error(pi_5, v_5)

In [ ]:
print(f_5.item(), f0_5.item())

3.2175827026367188 3.8429031372070312


In [ ]:
with torch.no_grad():
  nn.init.zeros_(game_b5.v)
  game_b5.v.data.add_(v0_5) 

In [ ]:
game_b5 = copy_game(game_b4)
with torch.no_grad():
  nn.init.zeros_(game_b5.v)
  game_b5.v.data.add_(v0_4) 

In [ ]:
game_b6, rho = optimize_game(game_b5, 1.0, n_epochs=30, verbose=False)

Epoch: 0, f: 3.59588e+00, delta f: -6.42807e+00%, nash satisfied: True, max g: -1.999e-02, dual nash satisfied: False, max product: 4.971e+00, rho: 9.249e-05, norm2 d0:3.221e-02
Epoch: 0, >0 satisfied: True, max g>0: -1.144e-03, dual >0 satisfied: False, max product >0: 4.398e-02, =1 satisfied: True, max g=1: -3.001e-04, dual =1 satisfied: False, max product =1: 2.925e-02
Epoch: 1, f: 3.40157e+00, delta f: -1.14843e+01%, nash satisfied: True, max g: -1.800e-02, dual nash satisfied: False, max product: 4.966e+00, rho: 9.249e-05, norm2 d0:2.727e-02
Epoch: 1, >0 satisfied: True, max g>0: -5.813e-04, dual >0 satisfied: False, max product >0: 4.392e-02, =1 satisfied: True, max g=1: -2.753e-04, dual =1 satisfied: False, max product =1: 2.633e-02
Epoch: 2, f: 3.22891e+00, delta f: -1.59773e+01%, nash satisfied: True, max g: -1.630e-02, dual nash satisfied: False, max product: 4.967e+00, rho: 9.249e-05, norm2 d0:2.490e-02
Epoch: 2, >0 satisfied: True, max g>0: -3.008e-04, dual >0 satisfied: Fa